In [ ]:
# default_exp noise.fault_generator

# Fault generator

> Parent class of generator of fault circuits according to specified error parameters

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
#export
from qsam.datatypes.circuit import Circuit
import numpy as np

In [ ]:
#export
DEPOLAR1 = {"X", "Y", "Z"}
DEPOLAR2 = {('I', 'X'), ('I', 'Y'), ('I', 'Z'),
            ('X', 'I'), ('X', 'X'), ('X', 'Y'), ('X', 'Z'),
            ('Y', 'I'), ('Y', 'X'), ('Y', 'Y'), ('Y', 'Z'),
            ('Z', 'I'), ('Z', 'X'), ('Z', 'Y'), ('Z', 'Z')}
XFLIP = {"X"}
ZFLIP = {"Z"}

# GATEROTS = {"MSd": (("X","X"), np.pi/2),
#             "CNOT": (("X","X"), np.pi/2),
#             "X": ("X", np.pi/2),
#             "Z": ("Z", np.pi/2),
#             "Y": ("Y", np.pi/2),
#             "H": ("XZ", np.pi/2),
            
#            }

In [ ]:
# export
class FaultGenerator:
    
    def __init__(self):
        self.GATE1_ERRSET = DEPOLAR1
        self.GATE2_ERRSET = DEPOLAR2
        self.MEAS_ERRSET = XFLIP
        self.INIT_ERRSET = XFLIP
        self.IDLE_ERRSET = DEPOLAR1
    
    @staticmethod
    def faults_from_weights(self, weights):
        faults = []
        for group,locs in par.items():
            r_ids = np.random.choice(len(locs), weights[group], replace=False)
            faults += [locs[r_idx] for r_idx in r_ids]
        return faults
    
    def gen_circuit(self, n_ticks, faults=[]):
        """Generate an empty circuit with given faults"""
        fault_circuit = Circuit([{} for _ in range(n_ticks)])
        for (tick_index,qubit) in faults:
            if isinstance(qubit, int):
                f_gate = np.random.choice(self.GATE1_ERRSET)
                qb_set = fault_circuit[tick_index].get(f_gate, set()) # get previous f_gate type fault
                qb_set.add(qubit) # append faults in this tick
                fault_circuit[tick_index][f_gate] = qb_set # udpate 
            elif isinstance(qubit, (list,set,tuple)):
                f_gates = self.GATE2_ERRSET[np.random.choice(len(self.GATE2_ERRSET))]
                for f_gate, qubit_i in zip(f_gates, qubit):
                    if f_gate != "I":
                        qb_set = fault_circuit[tick_index].get(f_gate, set())
                        qb_set.add(qubit_i)
                        fault_circuit[tick_index][f_gate] = qb_set
        return fault_circuit  

In [ ]:
class InnsbruckTrapFG(FaultGenerator):
    
    def __init__(self):
        
        # device parameters/gate application times in seconds
        tau = 1e-1  # Decoherence time 
        t1 = 15e-6  # single qubit gates
        t2 = 200e-6 # two-qubit gates
        tmeas = 300e-6 # measurement (gates)
        tinit = 0 # initialization
        delta = 1e-9 # Avg. gate overrotation angle 
        xt = True # enable crosstalk
        
        # possible errors in partitions
        gate1_errors = DEPOLAR1
        gate2_errors = DEPOLAR2
        meas_errors = XFLIP
        init_errors = XFLIP
        idle_errors = DEPOLAR1
        
        super().__init__(t1=t1,t2=t2,tmeas=tmeas,tinit=tinit,tau=tau,delta=delta)

NameError: name 'FaultGenerator' is not defined